In [ ]:
import numpy
import random

In [ ]:
#Here is code to generate and manipulate petal diagrams

def InsertTrivialPetal(petal, record):
    #Currently also edits the original petal, oh well
    loc = random.randrange(1, len(petal)+1)  #Should the insertion be able to happen in front of the start position?
    value = random.randrange(1, max(petal)+2) #Should the value be able to be 0? (+2 because need +1 and +1 because limit not inclusive)
    valueString = ""
    for k in range(len(petal)):
        if petal[k] >= value:
            petal[k] += 2
    new = [o for o in petal[:loc]]
    if random.random() < 0.5:
        new.append(value)
        new.append(value+1)
        valueString = str(value) + " " + str(value+1)
    else:
        new.append(value+1)
        new.append(value)
        valueString = str(value+1) + " " + str(value)
    for p in petal[loc:]:
        new.append(p)
    # print(new)
    record += "Inserted " + valueString + " at " + str(loc) + ", "
    return new, record

def DeleteTrivialPetal(petal, loc, record):
    new = petal.copy()
    p1 = petal[loc]
    p2 = petal[loc+1]
    new.remove(p1)
    new.remove(p2)
    for k in range(len(new)):
        if (new[k] > p1) or (new[k] > p2):
            new[k] -= 2
    # print(new)
    record += "Deleted at " + str(loc) + ", "
    return new, record

def CrossingExchange(petal, mloc, wloc, mplusloc, wplusloc, record):
    new = petal.copy()
    new[mloc] = petal[mplusloc]
    new[wloc] = petal[wplusloc]
    new[mplusloc] = petal[mloc]
    new[wplusloc] = petal[wloc]
    # print(new)
    record += ( "Replaced " + str(petal[mloc]) + ", " + str(petal[mplusloc]) + " at " + str(mloc) + " and " + str(petal[wloc]) + ", " + str(petal[wplusloc]) + " at " + str(wloc) )
    return new, record


def SimplifyPetalDiagram(petal, crossingExchangesAllowed = True):
    record = ''
    runsWithoutAMove = 0

    for i in range(1000):
        madeamove = False
        randomStart = random.randint(1,len(petal))
        if random.random() < 0.5:
            for j in range(-randomStart, len(petal)-1 -randomStart):
                if (petal[j] - petal[j+1])**2 == 1:
                    petal,record = DeleteTrivialPetal(petal, j, record)
                    madeamove = True
                    break
        elif crossingExchangesAllowed:
            for j in range(-randomStart, len(petal)-1 -randomStart):
                m = petal[j]
                w = petal[j+1]
                for k in range( j+2,len(petal)-1 -randomStart ):
                    if (k-j)%2 == 1:
                        continue
                    # print(petal[j+2:])
                    canExchange = False
                    if (petal[k] == m+1 and petal[k+1] == w-1):
                        # print(k-j)
                        canExchange = True
                        for l in range(j+2,k):
                            if (petal[l] > m+1 and petal[l] < w-1) or (petal[l] < m and petal[l] > w ):
                                canExchange = False  #Need some sort of checking the other direction code later
                        if canExchange:
                            petal, record = CrossingExchange(petal, j, k+1, k, j+1, record)
                            # madeamove = True
                            break

                    if (petal[k] == m-1 and petal[k+1] == w+1):
                        # print(k-j)
                        canExchange = True
                        for l in range(j+2,k):
                            if (petal[l] > m and petal[l] < w) or (petal[l] < m-1 and petal[l] > w+1 ):
                                canExchange = False
                        if canExchange:
                            petal, record = CrossingExchange(petal, k, j+1, j, k+1, record)
                            # madeamove = True
                            break
                    if canExchange:
                        break
        if madeamove:
            runsWithoutAMove = 0
        else:
            runsWithoutAMove += 1
            if runsWithoutAMove > 20:
                break
    return petal, record

def RandomlyComplicatePetal(petal, repeats, exchangesAllowed = True, record = ''):
    for i in range(repeats):
        randomStart = random.randint(0,len(petal)-2)
        if random.random() < 0.5 and exchangesAllowed:
            for j in range(-randomStart,len(petal)-1 - randomStart):   #Should this be able to cross exchange with the starting number?
                m = petal[j]
                w = petal[j+1]
                for k in range( j+2,len(petal)-1 ):
                    if (k-j)%2 == 1:
                        continue
                    canExchange = False
                    if (petal[k] == m+1 and petal[k+1] == w-1):
                        canExchange = True
                        for l in range(j+2,k):
                            if (petal[l] > m+1 and petal[l] < w-1) or (petal[l] < m and petal[l] > w ):
                                canExchange = False
                                #Need some sort of checking the other direction code later
                        if canExchange:
                            petal, record = CrossingExchange(petal, j, k+1, k, j+1, record)
                            break

                    if (petal[k] == m-1 and petal[k+1] == w+1):
                        canExchange = True
                        for l in range(j+2,k):
                            if (petal[l] > m and petal[l] < w) or (petal[l] < m-1 and petal[l] > w+1 ):
                                canExchange = False
                        if canExchange:
                            petal, record = CrossingExchange(petal, k, j+1, j, k+1, record)
                            break
                if canExchange:
                    break
        else:
            petal, record = InsertTrivialPetal(petal, record)
    return petal, record

In [ ]:
#Here is the list of possible invariants we developed
#We moved these to Mathematica to test all possible linear combinations because we also
#wanted to include graph characteristics and to test them against classical invariants

Gap = lambda x: sum([abs(x[i-1] - x[i]) for i in range(len(x))])
GapMinus = lambda x: sum([abs(x[i-1] - x[i]) -1 for i in range(len(x))])


def PetalToStem(petal, loc = -1, value = -1):
    #Currently also edits the original petal, oh well
    if loc == -1:
        loc = random.randrange(1,len(petal)+1)  #Should the insertion be able to happen in front of the start position?
    if value == -1:
        value = random.randrange(1, max(petal)+2) #Should the value be able to be 0?

    for k in range(len(petal)):
        if petal[k] >= value:
            petal[k] += 1
    new = [o for o in petal[:loc]]
    new.append(value)
    for p in petal[loc:]:
        new.append(p)
    # print(new)
    # record += "Inserted at " + str(loc) + ", "
    return new #, record


def Crossings(stem):
    """ Input a stem diagram to get the number of crossings """
    crossingLeft, crossingRight = 0,0
    for i in range(len(stem)):
        if stem[i] > stem[i-1]:
            n = stem[i]
            m = stem[i-1]
        else:
            m = stem[i]
            n = stem[i-1]
        for j in range(i%2, len(stem), 2):
            # print(j)
            if (stem[j] > stem[j-1] ):
                p = stem[j-1]
                q = stem[j]
            else:
                q = stem[j-1]
                p = stem[j]

            if (m < p) and (m < q) and (n > p) and (n < q):
                if i%2 == 0: #Might need to flip these
                    crossingRight += 1
                else:
                    crossingLeft += 1
    return [crossingLeft, crossingRight, crossingLeft + crossingRight]

def Nestings(stem):
    """ Input a stem diagram to get the number of nestings """
    nestingLeft, nestingRight = 0, 0
    for i in range(len(stem)):
        if stem[i] > stem[i-1]:
            n = stem[i]
            m = stem[i-1]
        else:
            m = stem[i]
            n = stem[i-1]
        for j in range(i%2, len(stem), 2):
            if (stem[j] > stem[j-1] ):
                p = stem[j-1]
                q = stem[j]
            else:
                q = stem[j-1]
                p = stem[j]

            if (m < p) and (m < q) and (n > p) and (n > q):
                if i%2 == 0: #Might need to flip these
                    nestingRight += 1
                else:
                    nestingLeft += 1

    return [nestingLeft, nestingRight, nestingLeft+nestingRight]

def LeftAndRightGap(stem):
    return [sum([abs(stem[i-1] - stem[i]) for i in range(len(stem)) if i%2 == 1]), sum([abs(stem[i-1] - stem[i]) for i in range(len(stem)) if i%2 == 0])]

def Height(petal):
    max = 0
    for i in range(len(petal)):
        crossings = 0
        for j in range(len(petal)):
            if petal[j] > petal[i] and j < i:
                crossings += 1
            elif petal[j] < petal[i] and j > i:
                crossings += 1
        if crossings > max:
            max = crossings
    return max

def TotalPermutationCrossings(petal):
    crossings = 0
    for i in range(len(petal)):
        for j in range(len(petal)):
            if petal[j] > petal[i] and j < i:
                crossings += 1
            elif petal[j] < petal[i] and j > i:
                crossings += 1

    return crossings // 2

def EvenOddGap(petal):
    even = ([(petal[i-1] - petal[i]) for i in range(1,len(petal)) if i%2 == 0])
    odd = ([(petal[i-1] - petal[i]) for i in range(len(petal)) if i%2 == 1])
    odd.append(petal[-1] - petal[0])

    return odd, even, sum(odd), sum(even)

def EvensMinusOdds(petal):
    return sum([ petal[i]*(-1)**i  for i in range(len(petal))])

def NonAdjacent(petal):
    return sum([ 1 for i in range(len(petal)) if (petal[i] - petal[i-1])**2 != 1  ])

def DistanceToCountThroughPermutation(petal):
    distance = 0
    for i in range(len(petal)):
        for j in range(1,len(petal)):
            if petal[(i+j)%len(petal)] == (petal[i]+1)%(len(petal)):  #Remove mod len petal if you don't want to measure the last distance
                distance += j
                break
    return distance

def DistanceToCountThroughPermutationBackwards(petal):
    return DistanceToCountThroughPermutation(Reverse(petal))

def Ascents(petal):
    return sum([1 for i in range(len(petal)) if (petal[i] - petal[i-1] > 0)])

def Descents(petal):
    return sum([1 for i in range(len(petal)) if (petal[i] - petal[i-1] < 0)])

def Records(petal):
    records = 1
    high = petal[0]
    for i in range(1,len(petal)):
        if petal[i] > high:
            high = petal[i]
            records += 1
    return records

def HighestLowestDistance(petal):
    for i in range(len(petal)):
        if petal[i] == 0:
            indexLow = i
            break
    for i in range(len(petal)):
        if petal[i] == len(petal)-1:
            indexHigh = i
            break
    return [indexHigh - indexLow, indexHigh, indexLow]

def EvenDistanceToCount(petal):
    rounds = 0
    currentEven = 2
    for i in range(len(petal)):
        if petal[i] == 0:
            indexLow = i
            break
    for i in range(len(petal)):
        rounds += 1
        for j in range(len(petal)):
            # print(i,j,petal[(j+indexLow)%len(petal)], currentEven)
            if petal[(j+indexLow)%len(petal)] == currentEven:
                currentEven += 2
        if currentEven >= len(petal)+1:
            break
    return rounds

def OddDistanceToCount(petal):
    rounds = 0
    currentOdd = 1
    for i in range(len(petal)):
        if petal[i] == 1:
            indexLow = i
            break
    for i in range(len(petal)):
        rounds += 1
        for j in range(len(petal)):
            # print(i,j,petal[(j+indexLow)%len(petal)], currentEven)
            if petal[(j+indexLow)%len(petal)] == currentOdd:
                currentOdd += 2

        if currentOdd >= len(petal):
            break
    return rounds

def OrderVector(petal):
    """This doesn't do anything useful"""
    doublePetal = (petal.copy())
    doublePetal.extend(petal.copy())  #Okay, is this -1 below this comment important?
    return [DistanceToCountThroughPermutation( doublePetal[i:i+len(petal)-1]  )//len(petal) for i in range(len(petal))]

def TotalCrossingsVector(petal):
    doublePetal = (petal.copy())
    doublePetal.extend(petal.copy())
    # print(doublePetal[2:2+len(petal)])
    return [TotalPermutationCrossings( doublePetal[i:i+len(petal)]  ) for i in range(len(petal))]

def ReduceGapParity(petal):
    GapParityList = [(petal[i-1] - petal[i])%3 for i in range(len(petal))]
    sorted = False
    while (not sorted):
        for i in range(len(GapParityList)):
            if (GapParityList[i-1] == 1) and (GapParityList[i] == 1):
                del GapParityList[i]
                del GapParityList[i-1]
                print(GapParityList)
                break
            elif (GapParityList[i-2] == 0) and (GapParityList[i-1] == 1) and (GapParityList[i] == 0):
                del GapParityList[i]
                del GapParityList[i-2]
                print(GapParityList)
                break
        sorted = True
        for i in GapParityList:
            if (i == 1):
                sorted = False
    return GapParityList

def ReduceGapFive(petal):
    GapParityList = [(petal[i-1] - petal[i])%5 for i in range(len(petal))]
    print(GapParityList)
    zeros = len( [i for i in GapParityList if i == 0] )%2
    ones = len( [i for i in GapParityList if i == 1] )%2
    twos = len( [i for i in GapParityList if i == 2] )%2
    threes = len( [i for i in GapParityList if i == 3] )%2
    fours = len( [i for i in GapParityList if i == 4] )%2
    return zeros, ones, twos, threes, fours

def CircleCrossings(petal):
    crossings = 0
    for i in range(len(petal)):
        if petal[i] > petal[i-1]:  #a < b
            a = petal[i-1]
            b = petal[i]
        else:
            a = petal[i]
            b = petal[i-1]

        for j in range(i-len(petal)+1, i):
            if petal[j] > petal[j-1]:  #m < n
                m = petal[j-1]
                n = petal[j]
            else:
                m = petal[j]
                n = petal[j-1]
            if (a != m and b != n):
                if (a < m and m < b and not (a<n and n<b)) or (a < n and n < b and not (a<m and m<b)):
                    # print(a,b,m,n)
                    crossings += 1
    if crossings%2 != 0:
        print("CircleCrossings had an odd number when it meant to double count")
    return crossings//2

def AlteredTotalCrossings(petal):
    alteredPetal = []
    for i in range(len(petal)):
        for j in range(len(petal)):
            if petal[-j] == i:
                alteredPetal.append(petal[-j+1])
    # print(alteredPetal)
    return TotalPermutationCrossings(alteredPetal)

def MaxCrossing(petal):
    #Oops, same as height
    max = 0
    for i in range(len(petal)):
        crossings = 0
        for j in range(len(petal)):
            if petal[j] > petal[i] and j < i:
                crossings += 1
            elif petal[j] < petal[i] and j > i:
                crossings += 1
        if crossings > max:
            max = crossings

    return max  #Need to divide by 2?

def AlteredMaxCrossing(petal):
    alteredPetal = []
    for i in range(len(petal)):
        for j in range(len(petal)):
            if petal[-j] == i:
                alteredPetal.append( petal[-j+1] )
    print(alteredPetal)
    return MaxCrossing(alteredPetal)

def FixedPointVector(petal):
    doublePetal = (petal.copy())
    doublePetal.extend(petal.copy())
    return [sum( [1 for j in range(len(doublePetal[i:i+len(petal)])) if j == doublePetal[i:i+len(petal)][j] ]) for i in range(len(petal))]

def PermutationCrossingsVector(petal):
    """ This will count how many crossings are on each line """
    crossingsVector = []
    for i in range(len(petal)):
        crossings = 0
        for j in range(len(petal)):
            if petal[j] > petal[i] and j < i:
                crossings += 1
            elif petal[j] < petal[i] and j > i:
                crossings += 1
        crossingsVector.append(crossings)

    return crossingsVector


def NonParallelLines(petal):
    slopes = [i-petal[i] for i in range(len(petal))]
    slopeSum = sum([abs(i) for i in slopes])
    print(slopes)
    lines = 0
    marked = []
    if slopes[-1] - slopes[0] == len(petal):
        new = slopes[1:len(slopes)-1]
        slopes = new

    for i in range(len(slopes)):
        if slopes[i] in marked:
            continue
        count = 0
        for j in range(len(slopes)):
            if slopes[j] == slopes[i]:
                count += 1
        if count%2 == 1:
            marked.append(slopes[i])
    return slopeSum, ((len(marked)-1)/2)%2

def LinesWithoutMatchingCrossingVectors(petal):
    identity = range(len(petal))
    crossingVectorList = []
    #Get the crossing vectors in a list
    for i in identity:
        crossings = []
        for j in range(len(petal)):
            if petal[j] > petal[i] and j < i:
                crossings.append(petal[j])
            elif petal[j] < petal[i] and j > i:
                crossings.append(petal[j])
        crossings.sort()
        crossingVectorList.append(crossings)
    # print(crossingVectorList)
    #Now loop through removing pairs of duplicates
    sorted = False
    replacer = -1
    length = len(petal)
    count = length
    while not sorted:
        sorted = True
        for i in identity:
            for j in identity:
                if i != j:
                    copy1 = crossingVectorList[i].copy()
                    copy1.append(petal[i])
                    copy1.sort()
                    copy2 = crossingVectorList[j].copy()
                    copy2.append(petal[j])
                    copy2.sort()
                    if crossingVectorList[i] == crossingVectorList[j] or copy1 == copy2:
                        crossingVectorList[i] = [replacer]
                        crossingVectorList[j] = [replacer-1]
                        replacer -= 2
                        count -= 2
                        sorted = False
                        for k in identity:
                            if petal[i] in crossingVectorList[k]:
                                crossingVectorList[k].remove(petal[i])
                            if petal[j] in crossingVectorList[k]:
                                crossingVectorList[k].remove(petal[j])


    # print(crossingVectorList)
    return count

def TotalCrossingsBackwards(petal):
    return TotalPermutationCrossings(Reverse(petal))

def CountByTwo(petal):
    """Returns the petal counted by two"""
    extended = petal.copy()
    extended.extend(petal.copy())
    return [extended[i] for i in range(len(extended)) if i%2 == 0]

def Reverse(petal):
    newPetal = petal.copy()
    newPetal.reverse()
    return newPetal

def TotalCrossingsByTwo(petal):
    return TotalPermutationCrossings(CountByTwo(petal))

def ParityChanges(petal):
    return sum([1 for i in range(len(petal)) if (petal[i]-petal[i-1])%2 == 1  ])

def CrossingsByTwoBackwards(petal):
    return TotalCrossingsByTwo(Reverse(petal))

def HigherToTheRight(petal):
    higherVector = []
    for i in range(len(petal)):
        higherCount = 0
        for j in range(i,len(petal)):
            if i != j:
                if petal[j] > petal[i]:
                    higherCount += 1
        higherVector.append(higherCount)
    return higherVector

def LowerToTheRight(petal):
    higherVector = []
    for i in range(len(petal)):
        higherCount = 0
        for j in range(i,len(petal)):
            if i != j:
                if petal[j] < petal[i]:
                    higherCount += 1
        higherVector.append(higherCount)
    return higherVector

def EvenCount(petal):
    return sum([1 for i in range(len(petal)) if petal[i]%2 == 0])

def Order(petal):
    return DistanceToCountThroughPermutation(petal)//len(petal)

def LinesWithOddCrossings(petal):
    crossingsVector = PermutationCrossingsVector(petal)
    return sum([1 for i in range(len(crossingsVector)) if crossingsVector[i]%2 == 1])

def EvensAtEvenLocations(petal):
    return sum([1 for i in range(len(petal)) if (petal[i]%2 == 0  and i%2 == 0) ])

def OddsAtOddLocations(petal):
    return sum([1 for i in range(len(petal)) if (petal[i]%2 == 1  and i%2 == 1) ])

def SimplifyTwoCount(petal):
    twoCount = CountByTwo(petal)
    odds = twoCount[:(len(twoCount)+1)//2]
    evens = twoCount[(len(twoCount)+1)//2:]
    # Maybe don't need this min, need to resize after removal
    return min([RecursiveRemovalOnOddEvenPairing(odds, evens), RecursiveRemovalOnOddEvenPairing(evens, odds)])

def RecursiveRemovalOnOddEvenPairing(odds, evens):
    #Works for in betweens on evens, need to check odds also
    for i in range(len(odds)):
        removalPairs = []
        for j in range(len(evens)):
            if (odds[i]-evens[j])**2 == 1:
                # print("removed")
                removalPairs.append([odds[i], evens[j]])
        if len(removalPairs) > 1:
            firstOdd = odds.copy()
            firstOdd.remove(removalPairs[0][0])
            firstEven = evens.copy()
            firstEven.remove(removalPairs[0][1])
            secondOdd = odds.copy()
            secondOdd.remove(removalPairs[1][0])
            secondEven = evens.copy()
            secondEven.remove(removalPairs[1][1])
            return min([RecursiveRemovalOnOddEvenPairing(firstOdd,firstEven), RecursiveRemovalOnOddEvenPairing(secondOdd, secondEven) ])
        if len(removalPairs) == 1:
            firstOdd = odds.copy()
            firstOdd.remove(removalPairs[0][0])
            firstEven = evens.copy()
            firstEven.remove(removalPairs[0][1])
            return RecursiveRemovalOnOddEvenPairing(firstOdd, firstEven)

    # print(odds,evens)
    return len(odds) + len(evens)

def LesserEvenPairs(petal):
    """This actually calculates it as if the petal inserted was 1-indexed"""
    newPetal = [i+1 for i in petal]
    return sum([1 for i in range(len(newPetal)-1) if (i%2 == 0 and newPetal[i] + newPetal[i+2] < newPetal[i+1])  ])

def LesserPairs(petal):
    """This actually calculates it as if the petal inserted was 1-indexed"""
    newPetal = [i+1 for i in petal]
    return sum([1 for i in range(len(newPetal)-2) if (newPetal[i] + newPetal[i+2] < newPetal[i+1])  ])

def JumpsUpOrDown(petal):
    """Code this one later"""

def MinusOneToTheRight(petal):
    """If defined slightly more carefully, this equals order."""
    return sum([1 for i in range(len(petal)) if petal[i]-1 in petal[i:]])

def HigherLowerRandomWalk(petal):
    return Ascents(petal) - Descents(petal)

def SkipOrder(petal):
    """Return a vector of order with each element removed sequentially"""
    return 0

def CountEvenDistances(petal):
    doublePetal = (petal.copy())
    doublePetal.extend(petal.copy())
    count = 0
    for i in range(len(petal)):
        for j in range(i, i+len(petal)):
            if doublePetal[j] == (petal[i] + 1)%len(petal):
                # print(petal[i], doublePetal[j], i, j)
                count += (j-i+1)%2
                # print(count)
    return count
def CountOddDistances(petal):
    doublePetal = (petal.copy())
    doublePetal.extend(petal.copy())
    count = 0
    for i in range(len(petal)):
        for j in range(i, i+len(petal)):
            if doublePetal[j] == (petal[i] + 1)%len(petal):
                # print(petal[i], doublePetal[j], i, j)
                count += (j-i)%2
                # print(count)
    return count

def Containments(petal):
    """Counts pairs that are contained by the adjacent numbers"""
    # return sum([1 for i in range(len(petal)) if ((petal[i-1] < petal[i] and petal[i-3] < petal[i-2] and petal[i-2] < petal[i] and petal[]) and ())])

def EqualizationsHigherLowerWalk(petal):
    count = 0
    location = 0
    for i in range(len(petal)):
        if petal[i-1] < petal[i]:
            location += 1
        else:
            location -= 1
        if location == 0:
            count += 1
    return count

def EqualizationsParityWalk(petal):
    count = 0
    location = 0
    for i in range(len(petal)):
        location += 2*(petal[i]%2)-1
        # print(location)
        if location == 0:
            count += 1
    return count

def LeadChangesParityWalk(petal):
    count = 0
    location = [0]
    for i in range(len(petal)):
        location.append( location[i] + 2*(petal[i]%2)-1 )
        # print(location)
        if (location[i] == 0 and location[i+1] == -1) or (location[i] == -1 and location[i+1] == 0):
            count += 1
    return count

def LeadChangesHigherLowerWalk(petal):
    count = 0
    location = [0]
    for i in range(len(petal)):
        if petal[i-1] < petal[i]:
            location.append(location[i] + 1)
        else:
            location.append( location[i] -1 )
        # print(location)
        if (location[i] == 0 and location[i+1] == -1) or (location[i] == -1 and location[i+1] == 0):
            count += 1
    return count

#Next idea, make a lead changes vector, be more careful about how you start


In [ ]:
Gap = lambda x: sum([abs(x[i-1] - x[i]) for i in range(len(x))])

distinguished = 0
distinguishedFromUnknot = 0
runs = 10000
length = 47
unknot = [i for i in range(length+2)]
minGap = Gap(unknot)
declaredUnknots = []
unknotsFound = []
errors = 0


for i in range(runs):
    petal1 = numpy.random.permutation(length)
    petal2 = numpy.random.permutation(length)
    Gap1 = Gap(petal1)
    Gap2 = Gap(petal2)
    equal = False  #Remove if checking for errors
    # if (minGap < Gap1 + 3):
    #     distinguishedFromUnknot += 1
    # if (minGap < Gap2 + 3):
    #     distinguishedFromUnknot += 1
    # else:
    #     declaredUnknots.append(petal2)

    # if (len(SimplifyPetalDiagram(petal2)[0]) == 1):
    #     unknotsFound.append(petal2)

    if (Gap1 < Gap2):
        Gap1 = Gap(numpy.append(petal1, [length, length+1]))
        if (Gap1 < Gap2 +3 -4*(length//6)):
            # simple1 = SimplifyPetalDiagram(petal1)[0]
            # simple2 = SimplifyPetalDiagram(petal2)[0]
            # equal = True
            # if (len(simple1) != len(simple2)):
            #     equal = False
            # else:
            #     for i in range(len(simple1)):
            #         if simple1[i] != simple2[i]:
            #             equal = False
            #             break
            if (equal):
                errors += 1
                print(petal1)
                print(petal2)
            else:
                distinguished += 1
    else:
        Gap2 = Gap(numpy.append(petal2, [length, length+1]))
        if (Gap2 < Gap1 +3 -4*(length//6)):
            # simple1 = SimplifyPetalDiagram(petal1)[0]
            # simple2 = SimplifyPetalDiagram(petal2)[0]
            # equal = True
            # if (len(simple1) != len(simple2)):
            #     equal = False
            # else:
            #     for i in range(len(simple1)):
            #         if simple1[i] != simple2[i]:
            #             equal = False
            #             break
            if (equal):
                errors += 1
                print(petal1)
                print(petal2)
            else:
                distinguished += 1
print(distinguished/runs)
print(distinguishedFromUnknot/(2*runs))
print(declaredUnknots)
print(errors)

#About 54% distinguishing with +3 at length 13
#About 54% distinguishing with + 4 at length 13, so maybe it isn't worth proofing, maybe isn't true (it looks like it might be a > +3 or a >= +4)
#63% with +3 at length 21
#82% with +3 at lenght 101

#Took 24 minutes, in 100,000 knots of length 13 it found 24,089 knots it could simplify to the unknot, none of which the gap function identified.

#Length 25, 65% distinguished correctly, with no errors
#Length 25, 58% distinguished correctly without + 3, no errors

#Length 25, 38% distinguished with -4*(length//7)
#Length 25, 38% distinguished with +3-4*(length//6)
#Length 47, 54% distinguished with +3-4*(length//6)
#Length 101, 66% distinguished with +3-4*(length//6)

0.5369
0.0
[]
0


In [ ]:
Gap = lambda x: sum([abs(x[i-1] - x[i]) for i in range(len(x))])

runs = 100
length = 25
changes = []

for i in range(runs):
    petal1 = numpy.random.permutation(length)
    petal2 = RandomlyComplicatePetal(petal1.copy(), 1)[0]
    Gap1 = Gap(petal1)
    Gap2 = Gap(petal2)
    changes.append(Gap2-Gap1)
    if ((Gap2 - Gap1) < 3 ):
        continue
        # print(Gap2 - Gap1)
        # print(petal1)
        # print(petal2)


print(changes)

#[4, 22, 14, 9, 7, 5, 15, 8, 24, 23, 17, 11, 10, 13,  0,  2, 12,  3, 16, 18, 20,  1, 19, 21, 6]
#[4, 22, 15, 8, 7, 5, 14, 9, 23, 24, 26, 25, 17, 11, 10, 13,  0,  2, 12,  3, 16, 18, 20,  1, 19, 21, 6]
#Performed an insertion and a crossing exchange and got a total gap change of 0

[20, 0, 0, 22, 16, 0, 46, 0, 30, 0, 24, 28, 42, 0, 0, 24, 20, 30, 0, 12, 0, 18, 16, 24, 26, -2, 0, 0, 2, 0, 0, 34, 0, 0, 0, 22, 2, 38, 20, 32, 30, 18, 0, 44, 0, 54, 0, 0, 24, 26, 16, -4, 20, 0, 24, 32, 32, 2, 26, 26, 0, -2, 22, 24, 0, 0, 26, 36, 18, 22, 26, 28, 20, 0, 24, 0, 0, 0, 0, 0, 0, 12, 0, -2, 40, -2, 40, 0, 30, 8, 0, 0, 32, 0, 6, 26, 0, 22, 26, -2]


In [ ]:
#Checking for a long loop that breaks the proposition
Gap = lambda x: sum([abs(x[i-1] - x[i]) for i in range(len(x))])
longRandomPermutation = [0, 55, 56, 117, 52, 88, 168, 62, 196, 73, 164, 107, 144, 36, 22, 61, 148, 45, 10, 105, 4, 66, 158, 145, 102, 159, 122, 100, 8, 31, 77, 170, 92, 29, 161, 189, 160, 124, 111, 5, 152, 138, 80, 134, 15, 114, 103, 2, 178, 19, 179, 187, 65, 126, 71, 115, 3, 47, 7, 59, 11, 57, 54, 162, 51, 93, 181, 87, 118, 23, 109, 123, 130, 150, 131, 43, 96, 81, 135, 193, 194, 166, 104, 200, 33, 184, 185, 34, 9, 175, 30, 195, 99, 16, 156, 116, 18, 50, 14, 21, 84, 90, 133, 60, 25, 91, 149, 39, 83, 82, 95, 177, 26, 32, 97, 165, 188, 75, 163, 67, 183, 157, 1, 129, 17, 20, 72, 147, 27, 38, 94, 13, 68, 172, 48, 197, 151, 86, 28, 140, 169, 78, 153, 46, 70, 139, 6, 141, 132, 35, 155, 125, 40, 69, 180, 136, 113, 53, 44, 98, 24, 106, 137, 64, 63, 199, 58, 192, 89, 12, 167, 176, 119, 41, 182, 121, 174, 173, 120, 42, 112, 128, 85, 76, 146, 198, 110, 37, 101, 108, 74, 186, 191, 190, 143, 171, 79, 49, 154, 127, 142]
# longRandomPermutation = [0,3,1,5,7,4,2,6,8]
for i in range(25):
    leave = False
    longRandomPermutation, des = RandomlyComplicatePetal(list(numpy.random.permutation( 201 )), 11)
    for i in des:
        if i == "p":
            print("Yes")
            leave = True
            print(longRandomPermutation)
            break
    if leave:
        break

In [ ]:
# longRandomPermutation = [105, 3, 183, 25, 51, 214, 161, 27, 28, 10, 85, 128, 42, 129, 39, 213, 168, 21, 54, 99, 74, 58, 66, 103, 121, 223, 65, 53, 159, 0, 29, 155, 208, 20, 166, 189, 102, 104, 110, 176, 84, 169, 86, 82, 24, 140, 146, 167, 209, 91, 13, 137, 119, 147, 92, 1, 49, 30, 207, 163, 95, 96, 141, 142, 158, 145, 153, 109, 203, 164, 196, 197, 212, 152, 69, 199, 118, 136, 144, 61, 117, 116, 186, 180, 201, 33, 219, 184, 100, 124, 62, 73, 72, 210, 45, 63, 179, 26, 170, 125, 48, 44, 38, 123, 211, 19, 41, 133, 138, 205, 114, 218, 14, 17, 35, 143, 40, 57, 221, 132, 89, 191, 157, 174, 94, 93, 15, 16, 187, 204, 106, 107, 7, 112, 67, 156, 139, 2, 98, 148, 22, 87, 206, 149, 175, 9, 220, 4, 5, 113, 154, 79, 173, 135, 55, 115, 43, 192, 151, 37, 12, 108, 101, 162, 18, 56, 97, 71, 81, 215, 111, 222, 200, 160, 172, 150, 34, 122, 190, 70, 181, 217, 127, 177, 126, 182, 120, 188, 78, 195, 50, 32, 83, 80, 224, 88, 6, 202, 60, 59, 130, 131, 76, 75, 77, 198, 23, 185, 64, 171, 36, 193, 11, 216, 31, 165, 47, 194, 46, 90, 134, 68, 52, 178, 8]

# print(longRandomPermutation)
# print(len(longRandomPermutation))

#[110, 55, 16, 43, 171, 75, 153, 173, 9, 160, 161, 115, 73, 22, 170, 15, 78, 45, 155, 5, 88, 19, 39, 49, 193, 13, 99, 192, 201, 77, 8, 107, 0, 194, 60, 114, 141, 46, 86, 163, 108, 133, 105, 187, 159, 178, 33, 56, 188, 132, 106, 74, 156, 150, 69, 72, 137, 67, 59, 118, 168, 27, 182, 124, 121, 94, 65, 14, 164, 111, 174, 144, 57, 36, 199, 92, 62, 42, 91, 89, 152, 66, 41, 138, 185, 131, 98, 4, 3, 100, 165, 143, 104, 142, 64, 26, 84, 180, 29, 76, 38, 135, 202, 120, 196, 177, 129, 195, 151, 186, 123, 37, 90, 79, 7, 28, 97, 24, 200, 47, 6, 113, 125, 63, 147, 204, 51, 148, 71, 102, 31, 197, 85, 48, 23, 40, 11, 61, 17, 20, 21, 82, 145, 109, 53, 117, 139, 146, 68, 25, 1, 175, 149, 140, 81, 136, 50, 130, 32, 183, 203, 18, 172, 2, 54, 119, 184, 112, 80, 181, 166, 52, 190, 128, 176, 12, 93, 83, 158, 179, 34, 35, 44, 191, 58, 70, 169, 30, 127, 10, 101, 198, 87, 126, 134, 96, 95, 157, 162, 167, 122, 103, 116, 189, 154]
#Results differ by 4

#[138, 109, 6, 60, 80, 5, 41, 121, 168, 146, 72, 188, 118, 141, 131, 104, 173, 197, 142, 191, 171, 178, 32, 62, 23, 13, 1, 149, 46, 45, 144, 78, 74, 37, 4, 58, 57, 83, 189, 79, 210, 3, 162, 202, 68, 17, 203, 101, 33, 164, 152, 155, 26, 64, 88, 44, 70, 40, 140, 182, 36, 183, 119, 190, 204, 206, 48, 82, 179, 151, 120, 166, 136, 61, 77, 76, 111, 161, 214, 63, 55, 10, 110, 31, 157, 102, 192, 135, 113, 124, 107, 25, 75, 133, 71, 174, 59, 116, 187, 184, 147, 156, 54, 186, 185, 27, 169, 96, 8, 7, 114, 125, 100, 65, 158, 209, 0, 15, 85, 94, 176, 212, 29, 132, 86, 28, 175, 130, 30, 84, 38, 106, 91, 2, 53, 50, 163, 90, 165, 213, 207, 97, 47, 112, 129, 99, 211, 205, 34, 200, 21, 127, 154, 117, 43, 208, 137, 180, 134, 24, 143, 148, 122, 115, 150, 194, 105, 139, 108, 19, 22, 199, 11, 12, 9, 49, 16, 89, 160, 69, 126, 153, 67, 18, 172, 198, 181, 167, 39, 66, 159, 103, 93, 92, 52, 51, 145, 14, 87, 177, 195, 95, 81, 170, 35, 42, 56, 201, 98, 128, 196, 20, 123, 193, 73]
#Original
#[126, 93, 6, 48, 66, 5, 37, 105, 156, 134, 60, 172, 102, 129, 119, 88, 161, 181, 130, 175, 159, 166, 28, 50, 19, 9, 1, 137, 132, 64, 62, 33, 4, 69, 173, 65, 194, 3, 150, 186, 56, 13, 187, 85, 29, 152, 140, 143, 22, 52, 74, 40, 58, 36, 128, 170, 32, 171, 103, 174, 188, 190, 42, 68, 167, 139, 104, 154, 124, 49, 95, 149, 198, 51, 45, 8, 94, 27, 145, 86, 176, 123, 97, 108, 91, 21, 63, 121, 59, 162, 47, 100, 135, 144, 44, 23, 157, 80, 98, 109, 84, 53, 146, 193, 0, 11, 71, 78, 164, 196, 25, 120, 72, 24, 163, 118, 26, 70, 34, 90, 77, 2, 151, 76, 153, 197, 191, 81, 41, 96, 117, 83, 195, 189, 30, 184, 17, 115, 142, 101, 39, 192, 125, 168, 122, 20, 131, 136, 106, 99, 138, 178, 89, 127, 92, 15, 18, 183, 7, 43, 12, 75, 148, 57, 113, 111, 114, 110, 112, 141, 55, 14, 160, 182, 169, 155, 35, 54, 147, 87, 133, 10, 73, 165, 179, 79, 67, 158, 31, 38, 46, 185, 82, 116, 180, 16, 107, 177, 61]
#Gap 12572, length 199, simplified without crossing exchanges
#[122, 93, 6, 48, 66, 5, 37, 105, 152, 130, 60, 168, 102, 125, 115, 88, 157, 177, 126, 171, 155, 162, 28, 50, 19, 9, 1, 133, 128, 64, 62, 33, 4, 69, 169, 65, 190, 3, 146, 182, 56, 13, 183, 85, 29, 148, 136, 139, 22, 52, 74, 40, 58, 36, 124, 166, 32, 167, 103, 170, 184, 186, 42, 68, 163, 135, 104, 150, 120, 49, 95, 145, 194, 51, 45, 8, 94, 27, 141, 86, 172, 119, 97, 108, 91, 21, 63, 117, 59, 158, 47, 100, 131, 140, 44, 23, 153, 80, 98, 109, 84, 53, 142, 189, 0, 11, 71, 78, 160, 192, 25, 116, 72, 24, 159, 114, 26, 70, 34, 90, 77, 2, 147, 76, 149, 193, 187, 81, 41, 96, 113, 83, 191, 185, 30, 180, 17, 111, 138, 101, 39, 188, 121, 164, 118, 20, 127, 132, 106, 99, 134, 174, 89, 123, 92, 15, 18, 179, 7, 43, 12, 75, 144, 57, 110, 137, 55, 14, 156, 178, 165, 151, 35, 54, 143, 87, 129, 10, 73, 161, 175, 79, 67, 154, 31, 38, 46, 181, 82, 112, 176, 16, 107, 173, 61]
#Gap 12176, length 195, simplified without crossing exchanges
#So if I find the right spot and add some crossings on the shorter one, it could be a counterexample
#[124, 95, 8, 50, 68, 5, 39, 107, 154, 132, 62, 170, 104, 127, 117, 90, 159, 179, 128, 173, 157, 164, 30, 52, 21, 11, 1, 6, 7, 135, 130, 66, 64, 35, 4, 71, 171, 67, 194, 3, 148, 186, 58, 15, 187, 87, 31, 150, 138, 141, 24, 54, 76, 42, 60, 38, 126, 168, 34, 169, 105, 172, 188, 190, 44, 70, 165, 137, 106, 152, 122, 51, 97, 147, 200, 182, 181, 53, 47, 10, 96, 29, 143, 88, 174, 121, 99, 110, 93, 23, 65, 119, 61, 160, 49, 102, 133, 142, 46, 25, 155, 82, 100, 111, 86, 55, 144, 193, 0, 13, 73, 80, 162, 198, 27, 118, 74, 26, 161, 116, 28, 72, 36, 92, 79, 2, 149, 78, 151, 199, 191, 83, 43, 98, 115, 85, 197, 196, 195, 189, 32, 184, 19, 113, 140, 103, 41, 192, 123, 166, 120, 22, 129, 134, 108, 101, 136, 176, 91, 125, 94, 17, 20, 183, 9, 45, 14, 77, 146, 59, 112, 139, 57, 16, 158, 180, 167, 153, 37, 56, 145, 89, 131, 12, 75, 163, 177, 81, 69, 156, 33, 40, 48, 185, 84, 114, 178, 18, 109, 175, 63]
#Gap 12268, length 201, just three trivial petals added to the one above

#[12, 7, 0, 16, 11, 8, 5, 4, 3, 18, 9, 15, 20, 21, 2, 1, 14, 17, 10, 6, 19, 13, 22]
#Great example of different tiered simplifications, gives 70-74 at length 13 and 82-88 at length 15

for k in range(1):
    for i in range(25):
        leave = False
        longRandomPermutation, des = RandomlyComplicatePetal(list(numpy.random.permutation( 15 )), 11)
        for i in des:
            if i == "p":
                # print("Yes")
                leave = True
                break
        if leave:
            break
    # longRandomPermutation = [22, 24, 23, 8, 10, 12, 5, 6, 25, 14, 13, 11, 26, 9, 7, 15, 16, 17, 18, 1, 19, 0, 4, 21, 20, 3, 2]
    longRandomPermutation = [14, 4, 6, 9, 7, 11, 15, 12, 10, 8, 16, 5, 3, 1, 13, 0, 2]
    # bigGap = Gap(longRandomPermutation)
    simpleList = []
    gapList = []
    lengthList = []
    reducedGapList = []        #Important Note: remember to remove this commenting when using several different base permutations
    reducedLengthList = []
    # descriptionList = []


    print("Base Permutation: " + str(longRandomPermutation))
    for i in range(100):
        tempPerm, des = RandomlyComplicatePetal(longRandomPermutation.copy(), 101)
        simple, description = SimplifyPetalDiagram(tempPerm, False)
        simple, trash = SimplifyPetalDiagram(simple.copy(), True)
        simpleList.append(simple)
        gapList.append( Gap(simple)  )

        lengthList.append(len(simple))
        # descriptionList.append(description)
        # if gapList[i] != gapList[0]:
        #     print (gapList[i])
        # if descriptionList[i] != descriptionList[0]:
        #     print (descriptionList[i])
    for i in range(len(gapList)):
        if (gapList[i] not in reducedGapList) or (lengthList[i] not in reducedLengthList):
            reducedGapList.append(gapList[i])
            reducedLengthList.append(lengthList[i])
    print(reducedGapList)
    print(reducedLengthList)



    for i in range(len(gapList)):
        for j in range(len(gapList)):
            if (gapList[i] >= gapList[j]) and (lengthList[i] < lengthList[j]):
                print(i)
                print(j)
                print(gapList[i])
                print(gapList[j])
                print(lengthList[i])
                print(lengthList[j])
                print(simpleList[i])
                print(simpleList[j])


# print(simpleList)
# print(gapList)
# print(lengthList)
# print(descriptionList)

#Ran 3500 times on  length 201 words randomly complicated 112 times, simplified without using crossing exchanges, couldn't find counterexample
#Ran 1600 times as above but used Xex on simplification (took 10 times as long)

#I guaranteed that these had potential for crossing exchanges

#Claim: A trivial petal insertion can only introduce the possibility of 1 new crossing exchange

#Note: this process does not always identify unknots
#[1, 24, 23, 5, 0, 26, 25, 14, 15, 19, 12, 3, 4, 13, 10, 7, 6, 9, 8, 11, 20, 18, 21, 17, 16, 22, 2]
#(0,1), (20-22,7), (36-38,9), (48,11) (62,13)

#Unfortunately loose results from
#[22, 24, 23, 8, 10, 12, 5, 6, 25, 14, 13, 11, 26, 9, 7, 15, 16, 17, 18, 1, 19, 0, 4, 21, 20, 3, 2]

#These are the lists of different gap and length combos I got from this very odd knot
#(46-48 and 52-56, 11),(58-60 and 64-76, 13), (74-90 and 98, 15), (94 and 98-108 and 112 and 120, 17), (120 and 132-136, 19) and trapped forms up to 27 length
# [46, 54, 68, 80, 60, 58, 86, 202, 66, 64, 78, 112, 98, 82, 48, 120, 128, 162, 100, 72, 70, 94, 134, 136, 76, 194, 102, 88, 232, 84, 218, 96, 138, 232, 56, 164, 90, 104, 74, 108, 52, 168]
# [11, 11, 13, 15, 13, 13, 15,  25, 13, 13, 15,  17, 15, 15, 11,  19,  19,  23,  17, 13, 13, 17,  19,  19, 15,  23,  17, 15,  25, 15,  25, 17,  21,  27, 11,  21, 15,  17, 13,  17, 11,  23]


#Very bad example, same knot if my code works
# [11, 2, 12, 5, 10, 3, 1, 8, 6, 0, 9, 7, 4]
# [12, 4, 8, 10, 13, 9, 14, 5, 7, 6, 3, 1, 11, 0, 2]
#Don't have the exact simplifications, but the above knot has overlap at 76 and 98 and 120

#Another bad example, from this same messed up knot
# [14, 4, 6, 9, 7, 11, 15, 12, 10, 8, 16, 5, 3, 1, 13, 0, 2]
#This has length 17 and gap 94, while we have definitely obtained a knot from this that had gap 98 with length 15
# [12, 6, 8, 10, 13, 9, 14, 7, 4, 2, 11, 0, 5, 1, 3]
# length 15, gap 74
# [16, 8, 10, 13, 11, 3, 5, 14, 17, 4, 6, 12, 18, 9, 7, 1, 15, 0, 2]
# [18, 6, 10, 13, 11, 15, 19, 16, 14, 12, 20, 9, 3, 1, 17, 0, 2, 8, 4, 5, 7]
# Gap 126 with length 19, Gap 122 with length 21


#Same as unknot?  Produced by complicating and simplifying
#[6, 3, 4, 2, 5, 1, 0]  Yes, just delete 3 times
#[0, 6, 3, 5, 1, 4, 2]  Yes, just Xex the 4,2 and 3,5 and then delete 3 times


Base Permutation: [14, 4, 6, 9, 7, 11, 15, 12, 10, 8, 16, 5, 3, 1, 13, 0, 2]
[94, 150, 172, 100, 46, 140, 102, 162, 120, 118, 116, 124, 96, 128, 106, 110, 114, 108, 178]
[17, 23, 21, 17, 11, 19, 17, 21, 19, 19, 19, 19, 17, 21, 19, 19, 19, 19, 23]
11
2
172
150
21
23
[18, 2, 4, 7, 5, 9, 19, 16, 8, 10, 15, 6, 20, 3, 14, 12, 17, 0, 13, 11, 1]
[18, 4, 6, 13, 7, 15, 19, 16, 14, 12, 21, 10, 8, 5, 3, 1, 9, 22, 20, 11, 17, 0, 2]
350
1651
140
128
19
21
[16, 2, 4, 7, 5, 9, 17, 14, 8, 6, 18, 3, 13, 11, 15, 0, 12, 10, 1]
[18, 6, 10, 13, 11, 15, 19, 16, 14, 12, 20, 7, 4, 2, 9, 0, 5, 8, 17, 1, 3]
350
3305
140
128
19
21
[16, 2, 4, 7, 5, 9, 17, 14, 8, 6, 18, 3, 13, 11, 15, 0, 12, 10, 1]
[18, 4, 6, 9, 7, 11, 19, 17, 15, 13, 10, 8, 20, 5, 3, 1, 16, 12, 14, 0, 2]
529
2
162
150
21
23
[18, 10, 12, 7, 2, 15, 13, 3, 5, 16, 19, 4, 6, 14, 20, 11, 9, 1, 17, 0, 8]
[18, 4, 6, 13, 7, 15, 19, 16, 14, 12, 21, 10, 8, 5, 3, 1, 9, 22, 20, 11, 17, 0, 2]


In [ ]:
length = len(longRandomPermutation)
maxGap = [0,0]
lengthDifference = [0,0]
for i in range(len(gapList)):
    for j in range(len(lengthList)):
        if lengthList[i] < lengthList[j]:
            if (gapList[i] > gapList[j]   ):  # +3-4*(length//6)  on the left
                # print(i)
                # print(j)
                # print(gapList[i])
                # print(gapList[j])
                # print(lengthList[i])
                # print(lengthList[j])
                if (gapList[i] - gapList[j] > maxGap[0] - maxGap[1]):
                    maxGap = [gapList[i], gapList[j]]
                    lengthDifference = [lengthList[i], lengthList[j]]
print(maxGap)
print(lengthDifference)
print(maxGap[0] - maxGap[1])
print()

[0, 0]
[0, 0]
0



In [ ]:
for i in range(len(gapList)):
    if gapList[i] == 12618:
        print(i)
        spot1 = i
    if gapList[i] == 12176:
        print(i)
        spot2 = i

print(gapList[spot1])
print(gapList[spot2])
print(lengthList[spot1])
print(lengthList[spot2])
# print(descriptionList[spot1])
# print(descriptionList[2])
print(simpleList[spot1])
print(simpleList[spot2])

20
12618
12432
195
193
[5, 49, 71, 28, 139, 29, 39, 150, 51, 47, 189, 140, 6, 159, 8, 190, 178, 23, 172, 64, 32, 162, 152, 174, 119, 180, 76, 12, 106, 181, 151, 130, 98, 30, 168, 1, 163, 56, 50, 53, 65, 133, 13, 102, 85, 94, 73, 147, 61, 70, 179, 45, 97, 156, 165, 185, 136, 108, 88, 95, 24, 143, 191, 161, 144, 14, 137, 42, 4, 43, 19, 187, 20, 138, 110, 84, 176, 69, 99, 78, 26, 164, 124, 67, 109, 107, 72, 93, 134, 173, 194, 66, 35, 125, 55, 31, 111, 153, 175, 170, 129, 127, 101, 115, 87, 74, 0, 117, 11, 48, 177, 149, 120, 16, 46, 141, 146, 59, 160, 41, 36, 60, 154, 21, 54, 122, 22, 104, 63, 27, 10, 116, 183, 123, 90, 3, 44, 188, 75, 167, 38, 34, 186, 2, 89, 83, 57, 131, 17, 15, 193, 128, 113, 182, 126, 9, 40, 79, 171, 121, 86, 158, 155, 169, 166, 18, 114, 148, 92, 81, 112, 145, 103, 68, 7, 118, 77, 135, 100, 184, 157, 52, 37, 142, 25, 105, 62, 132, 96, 192, 91, 82, 80, 33, 58]
[5, 49, 71, 28, 137, 29, 39, 148, 51, 47, 187, 138, 6, 157, 8, 188, 176, 23, 170, 64, 32, 160, 150, 172, 117, 1

In [ ]:
longRandomPermutation = [222, 103, 213, 16, 15, 92, 59, 101, 100, 195, 196, 191, 69, 198, 215, 169, 170, 1, 49, 168, 137, 78, 21, 30, 207, 2, 133, 55, 64, 178, 104, 0, 44, 187, 36, 5, 48, 107, 110, 76, 177, 189, 96, 218, 210, 115, 13, 68, 45, 43, 24, 163, 164, 34, 114, 184, 18, 197, 125, 29, 134, 80, 57, 128, 113, 108, 109, 124, 126, 10, 73, 132, 4, 27, 22, 208, 102, 227, 112, 138, 67, 143, 118, 117, 176, 188, 226, 201, 146, 33, 99, 205, 54, 214, 65, 23, 220, 47, 172, 77, 93, 140, 206, 58, 160, 35, 60, 154, 150, 165, 122, 66, 70, 37, 223, 148, 63, 19, 106, 129, 72, 56, 41, 94, 7, 209, 3, 50, 183, 20, 46, 38, 211, 105, 130, 12, 200, 199, 79, 83, 86, 194, 216, 173, 135, 136, 185, 144, 31, 52, 8, 225, 51, 116, 90, 204, 192, 6, 156, 157, 98, 120, 42, 53, 14, 88, 175, 153, 190, 74, 186, 95, 151, 212, 152, 40, 145, 89, 87, 82, 193, 202, 155, 123, 149, 158, 61, 17, 181, 182, 166, 91, 139, 39, 228, 97, 203, 32, 224, 147, 119, 221, 179, 141, 162, 71, 159, 217, 142, 127, 11, 84, 85, 171, 219, 111, 180, 28, 161, 121, 167, 81, 25, 26, 131, 9, 174, 75, 62]
happy = [22, 71, 113, 118, 16, 60, 59, 17, 0, 30, 83, 82, 73, 72, 104, 105, 86, 94, 43, 44, 80, 81, 29, 106, 103, 117, 79, 78, 42, 39, 116, 20, 58, 65, 46, 6, 7, 49, 64, 114, 115, 62, 61, 63, 11, 68, 112, 89, 90, 107, 69, 40, 75, 5, 8, 74, 41, 88, 87, 77, 111, 54, 53, 110, 84, 50, 45, 70, 14, 15, 92, 91, 67, 76, 93, 34, 109, 9, 10, 36, 37, 108, 33, 55, 23, 24, 25, 27, 28, 26, 85, 66, 1, 2, 52, 102, 99, 97, 96, 32, 31, 51, 101, 100, 21, 18, 19, 57, 56, 3, 4, 12, 13, 98, 95, 35, 38, 47, 48]


for i in range(len(longRandomPermutation)):
    for j in range(len(longRandomPermutation)):
        if (longRandomPermutation[i] == longRandomPermutation[j]) and (i != j):
            print(i)
            print(j)

for i in range(len(happy)):
    for j in range(len(happy)):
        if (happy[i] == happy[j]) and (i != j):
            print(i)
            print(j)

In [ ]:
num1 = [4, 8, 10, 2, 7, 1, 5, 3, 6, 0, 9]
num2 = [4, 8, 10, 2, 7, 1, 5, 3, 6, 11,12, 0, 9]
num3 = [4, 8, 10, 2, 11, 12, 7, 1, 5, 3, 6, 0, 9]
num4 = [10, 2, 7, 1, 5, 3, 6, 0, 9,4,8]

num1 = [0,2,7,4,6,9,1,3,8,5,10]
num2 = [0,3,6,1,7,9,2,4,8,5,10]
num3 = [0,3,1,5,7,4,2,6,8]

# num1 = [0,2,7,4,6,9,1,3,8,5,10]
# num2 = [0,3,6,1,7,9,2,4,8,5,10]
# num3 =

print(OddSum(num1) - EvenSum(num1))
print(OddSum(num2) - EvenSum(num2))
print(OddSum(num3) - EvenSum(num3))
print(OddSum(num4) - EvenSum(num4))
print([ OddSum(num1[i:len(num1)] + num1[:i] ) - EvenSum(num1[i:len(num1)] + num1[:i]) for i in range(len(num1)) ])
# print([num1[i:len(num1)] + num1[:i] for i in range(len(num1)) ])
print([ OddSum(num2[i:len(num2)] + num2[:i] ) - EvenSum(num2[i:len(num2)] + num2[:i]) for i in range(len(num2)) ])
print([ OddSum(num3[i:len(num3)] + num3[:i] ) - EvenSum(num3[i:len(num3)] + num3[:i]) for i in range(len(num3)) ])



-9
-11
0
-35
[-9, 9, -13, -1, -7, -5, -13, 11, -17, 1, -11]
[-11, 11, -17, 5, -7, -7, -11, 7, -15, -1, -9]
[0, 0, -6, 4, -14, 0, -8, 4, -16]


In [ ]:
petal1 = [3, 1, 4, 6, 8, 0, 2, 7, 5] #Almost simplified, but can Xex
# for i in range(1000000):
#     petal1 = list(numpy.random.permutation(17))
#     if ParityChanges(petal1) == 2:
#         print("success")
#         break
petal1, trash = SimplifyPetalDiagram(petal1, "")
print(petal1, ParityChanges(petal1), trash)
for i in range(1000):
    complicated, deets1 = RandomlyComplicatePetal(petal1.copy(), 10)
    simplified, deets2 = SimplifyPetalDiagram(complicated, "")
    if ParityChanges(simplified) == 2:
        shifted = petal1[1:]
        shifted.append(petal1[0])
        if simplified != petal1 and simplified != shifted:
            canXex = False
            for i in range(20):
                trash, deets = RandomlyComplicatePetal(simplified.copy(), 1)
                if "R" in deets:
                    canXex = True
            if not canXex:
                print(simplified, deets1, deets2)


[3, 1, 4, 6, 8, 0, 2, 7, 5] 2 
[2, 0, 4, 6, 8, 3, 1, 7, 5] Inserted 5 4 at 5, Inserted 11 10 at 6, Replaced 3, 4 at 0 and 0, 1 at 9Inserted 6 7 at 4, Replaced 3, 4 at 10 and 0, 1 at 1Replaced 3, 4 at 0 and 0, 1 at 11Inserted 5 4 at 5, Inserted 6 7 at 6,  Deleted at -7, Deleted at -4, Deleted at 5, Deleted at -8, Deleted at -7, 
[2, 0, 4, 6, 8, 3, 1, 7, 5] Inserted 5 4 at 5, Inserted 7 8 at 4, Inserted 9 10 at 12, Replaced 3, 4 at 0 and 0, 1 at 9Inserted 13 12 at 6, Inserted 3 2 at 1, Inserted 9 8 at 12, Inserted 14 15 at 7, Inserted 18 19 at 10, Inserted 3 2 at 12,  Deleted at -9, Deleted at -18, Deleted at -3, Deleted at 1, Deleted at 4, Deleted at 4, Deleted at 4, Deleted at -4, Deleted at -8, 
[5, 1, 6, 4, 2, 0, 3] Inserted 4 3 at 3, Replaced 2, 3 at 8 and 8, 9 at 5Inserted 3 4 at 5, Inserted 7 8 at 4, Replaced 4, 5 at -7 and 12, 13 at -2Replaced 4, 5 at -3 and 12, 13 at -6 Deleted at -11, Deleted at -9, Deleted at -6, Deleted at -2, 
[3, 1, 8, 4, 6, 0, 2, 5, 7] Inserted 4 5 at 3, I

In [ ]:
a = [16, 2, 14, 4, 10, 0, 12, 5, 3, 9, 13, 1, 15, 7, 11, 8, 6] #Cannot Xex
b = [16, 2, 14, 4, 8, 0, 12, 5, 3, 7, 13, 1, 15, 11, 9, 6, 10] #Cannot Xex
c = [2, 14, 4, 10, 0, 12, 5, 3, 9, 13, 1, 15, 7, 11, 8, 6, 16] #Cannot Xex
#These are all pseudo-simplified and generated from the same knot

petal1, deets = RandomlyComplicatePetal(c, 1)
print(petal1, deets)

[2, 16, 4, 12, 0, 14, 5, 3, 11, 10, 9, 15, 1, 17, 7, 13, 8, 6, 18] Inserted 11 10 at 8, 


In [ ]:
a = [10, 0, 5, 7, 3, 1, 9, 6, 4, 8, 2] #Can Xex
b = [8, 0, 4, 1, 7, 5, 3, 6, 2] #Can Xex
c = [0, 4, 1, 7, 5, 3, 6, 2, 8] #Can Xex
d = [1, 3, 0, 6, 4, 2, 5] #Can Xex
e = [6, 0, 3, 5, 1, 4, 2] #Can Xex
#These are all pseudo-simplified and generated from the same knot

petal1, deets = RandomlyComplicatePetal(e, 1)
print(petal1, deets)

[6, 0, 3, 4, 2, 5, 1] Replaced 4, 5 at -2 and 1, 2 at -3


In [ ]:
a = [0, 12, 6, 2, 9, 1, 15, 13, 11, 7, 5, 3, 16, 4, 8, 10, 14]  #Can Xex
b = [0, 10, 5, 3, 7, 1, 13, 11, 9, 2, 14, 4, 6, 8, 12] #Can Xex
c = [0, 13, 11, 9, 5, 3, 7, 1, 12, 2, 14, 4, 6, 8, 10] #Can Xex
d = [0, 6, 4, 1, 9, 7, 5, 3, 11, 2, 12, 10, 8] #Can Xex
e = [0, 5, 3, 1, 4, 2, 6] #Cannot Xex
#These are all pseudo-simplified and generated from the same knot

petal1, deets = RandomlyComplicatePetal(e, 1)
print(petal1, deets)

[0, 7, 3, 5, 4, 1, 6, 2, 8] Inserted 5 4 at 3, 


In [ ]:
for i in range(1):
    petal1 = list(numpy.random.permutation(17)) #[0,4,1,3,2]
    a = LinesWithoutMatchingCrossingVectors(petal1)
    petal2, deets = RandomlyComplicatePetal(petal1.copy(), 50)
    petal2, deets = SimplifyPetalDiagram(petal2.copy(), deets)
    # print(petal2, deets)
    b = LinesWithoutMatchingCrossingVectors(petal2)
    if "R" in deets:
        print(a,b)
        print(petal1)
        print(petal2, deets)

# print(LinesWithoutMatchingCrossingVectors([0, 6, 2, 4, 1, 5, 3]))
# print(LinesWithoutMatchingCrossingVectors([0, 6, 1, 5, 2, 4, 3]))

17 13
[16, 7, 4, 10, 14, 9, 11, 1, 12, 3, 8, 13, 5, 0, 2, 6, 15]
[4, 8, 12, 7, 9, 1, 10, 3, 6, 11, 5, 0, 2] Deleted at -51, Replaced 28, 29 at 16 and 30, 31 at -41Deleted at -21, Replaced 26, 27 at -40 and 28, 29 at 17Deleted at -40, Deleted at -31, Deleted at -15, Deleted at -22, Deleted at -55, Replaced 25, 26 at 1 and 23, 24 at 18Deleted at -38, Deleted at -21, Deleted at -29, Deleted at -24, Deleted at -33, Deleted at -36, Deleted at -8, Deleted at -11, Deleted at -27, Deleted at -3, Deleted at -18, Deleted at -27, Deleted at -2, Deleted at 1, Deleted at -20, Deleted at -9, Deleted at -8, Deleted at -1, Replaced 14, 15 at -6 and 12, 13 at 3Replaced 14, 15 at 2 and 12, 13 at -5Deleted at -6, Deleted at -1, Deleted at -14, 


In [ ]:
#[6, 2, 10, 1, 5, 3, 7, 9, 4, 0, 8]
#[5, 9, 6, 1, 0, 4, 8, 10, 3, 2, 7]
#[10, 11, 4, 0, 13, 5, 9, 3, 12, 14, 2, 6, 8, 1, 7]

# [14, 4, 6, 10, 8, 11, 15, 12, 7, 9, 16, 5, 3, 1, 13, 0, 2]  Same as Mr. Bad Knot?  Got it by messing around, but it only has 4 swapped and they don't seem likely

petal1 =  list(numpy.random.permutation(107))   #[0,3,1,4,2,6,5]
petal4, trash = SimplifyPetalDiagram(petal1.copy())
petal4 = [14, 4, 6, 9, 7, 11, 15, 12, 10, 8, 16, 5, 3, 1, 13, 0, 2]
petal4 = [0,1,2,3,4]
print(petal4)
# saved = TotalPermutationCrossings(petal4)
print(FixedPointVector(petal4))
# print( int(DistanceToCountThroughPermutation(petal1)/len(petal1)), TotalPermutationCrossings(petal1), Descents(petal1), EvenDistanceToCount(petal1))
# print(TotalPermutationCrossings(petal1), DistanceToCountThroughPermutation(petal1)//len(petal1), CircleCrossings(petal1), MaxCrossing(petal1), AlteredMaxCrossing(petal1))
# print(TotalPermutationCrossings(petal1)+ DistanceToCountThroughPermutation(petal1)//len(petal1)+ CircleCrossings(petal1))
for i in range(100):
    petal3, deets1 = RandomlyComplicatePetal(petal4.copy(), 1)#InsertTrivialPetal(petal1.copy(), "")
    # petal2, deets2 = SimplifyPetalDiagram(petal3.copy())
    petal2 = petal3

    # print( int(DistanceToCountThroughPermutation(petal2)/len(petal2)), TotalPermutationCrossings(petal2), Descents(petal2), EvenDistanceToCount(petal2))
    current = FixedPointVector(petal2)
    if (current != saved and deets1 != "") or (True and deets1 != ""):
        print(petal2, deets1)
        print(current)
        # print(TotalPermutationCrossings(petal2), DistanceToCountThroughPermutation(petal2)//len(petal2), CircleCrossings(petal2), MaxCrossing(petal2), AlteredMaxCrossing(petal2))
        # print(TotalPermutationCrossings(petal2) + DistanceToCountThroughPermutation(petal2)//len(petal2) + CircleCrossings(petal2))

    # print(DistanceToCountThroughPermutation(petal2)//len(petal2) + TotalPermutationCrossings(petal2))
    # if (HighestLowestDistance(petal2)[0] + HighestLowestDistance(petal2)[1] )%2 != saved:
    #      print(petal2, deets1)
    #      print(HighestLowestDistance(petal2))


#Checked Records, Gap, and Height, don't seem to play too well

[0, 1, 2, 3, 4]
[5, 0, 0, 0, 0]
[0, 1, 2, 5, 4, 3, 6] Inserted 4 3 at 4, 
[5, 0, 1, 0, 0, 1, 0]
[0, 1, 2, 5, 6, 4, 3] Inserted 4 3 at 5, 
[3, 1, 0, 1, 0, 2, 0]
[0, 3, 4, 5, 1, 2, 6] Inserted 1 2 at 4, 
[2, 0, 0, 2, 0, 3, 0]
[0, 1, 2, 5, 3, 4, 6] Inserted 3 4 at 4, 
[4, 2, 0, 0, 0, 1, 0]
[0, 1, 4, 5, 2, 3, 6] Inserted 4 5 at 2, 
[3, 0, 2, 0, 0, 2, 0]
[0, 1, 2, 3, 4, 5, 6] Inserted 2 3 at 2, 
[7, 0, 0, 0, 0, 0, 0]
[0, 1, 2, 3, 6, 4, 5] Inserted 4 5 at 5, 
[4, 2, 0, 0, 0, 1, 0]
[0, 1, 4, 2, 3, 5, 6] Inserted 2 3 at 3, 
[4, 2, 0, 0, 0, 1, 0]
[0, 1, 2, 4, 3, 5, 6] Inserted 4 3 at 3, 
[5, 1, 0, 0, 0, 0, 1]
[0, 1, 2, 4, 5, 3, 6] Inserted 4 5 at 3, 
[4, 0, 1, 0, 0, 0, 2]
[0, 3, 4, 5, 6, 1, 2] Inserted 1 2 at 5, 
[1, 0, 0, 0, 2, 4, 0]
[0, 5, 4, 1, 2, 3, 6] Inserted 5 4 at 1, 
[2, 0, 3, 1, 0, 1, 0]
[0, 1, 2, 5, 3, 4, 6] Inserted 3 4 at 4, 
[4, 2, 0, 0, 0, 1, 0]
[0, 1, 4, 2, 3, 5, 6] Inserted 2 3 at 3, 
[4, 2, 0, 0, 0, 1, 0]
[0, 1, 2, 5, 3, 4, 6] Inserted 3 4 at 4, 
[4, 2, 0, 0, 0, 1, 0]
[0, 1, 2

In [ ]:
stem1 = [1,8,6,4,2,5,7,3]
print(Crossings(stem1))
print(Nestings(stem1))
stem2 = [1,8,5,6,2,4,7,3]
print(Crossings(stem2))
print(Nestings(stem2))
stem3 = [1,4,8,6,2,5,7,3]
print(Crossings(stem3))
print(Nestings(stem3))
stem1 = list(numpy.random.permutation( 84 ))

# stem3 = [126, 93, 6, 48, 66, 5, 37, 105, 156, 134, 60, 172, 102, 129, 119, 88, 161, 181, 130, 175, 159, 166, 28, 50, 19, 9, 1, 137, 132, 64, 62, 33, 4, 69, 173, 65, 194, 3, 150, 186, 56, 13, 187, 85, 29, 152, 140, 143, 22, 52, 74, 40, 58, 36, 128, 170, 32, 171, 103, 174, 188, 190, 42, 68, 167, 139, 104, 154, 124, 49, 95, 149, 198, 51, 45, 8, 94, 27, 145, 86, 176, 123, 97, 108, 91, 21, 63, 121, 59, 162, 47, 100, 135, 144, 44, 23, 157, 80, 98, 109, 84, 53, 146, 193, 0, 11, 71, 78, 164, 196, 25, 120, 72, 24, 163, 118, 26, 70, 34, 90, 77, 2, 151, 76, 153, 197, 191, 81, 41, 96, 117, 83, 195, 189, 30, 184, 17, 115, 142, 101, 39, 192, 125, 168, 122, 20, 131, 136, 106, 99, 138, 178, 89, 127, 92, 15, 18, 183, 7, 43, 12, 75, 148, 57, 113, 111, 114, 110, 112, 141, 55, 14, 160, 182, 169, 155, 35, 54, 147, 87, 133, 10, 73, 165, 179, 79, 67, 158, 31, 38, 46, 185, 82, 116, 180, 16, 107, 177, 61]
# print(Crossings(stem3), Nestings(stem3))

#[122, 93, 6, 48, 66, 5, 37, 105, 152, 130, 60, 168, 102, 125, 115, 88, 157, 177, 126, 171, 155, 162, 28, 50, 19, 9, 1, 133, 128, 64, 62, 33, 4, 69, 169, 65, 190, 3, 146, 182, 56, 13, 183, 85, 29, 148, 136, 139, 22, 52, 74, 40, 58, 36, 124, 166, 32, 167, 103, 170, 184, 186, 42, 68, 163, 135, 104, 150, 120, 49, 95, 145, 194, 51, 45, 8, 94, 27, 141, 86, 172, 119, 97, 108, 91, 21, 63, 117, 59, 158, 47, 100, 131, 140, 44, 23, 153, 80, 98, 109, 84, 53, 142, 189, 0, 11, 71, 78, 160, 192, 25, 116, 72, 24, 159, 114, 26, 70, 34, 90, 77, 2, 147, 76, 149, 193, 187, 81, 41, 96, 113, 83, 191, 185, 30, 180, 17, 111, 138, 101, 39, 188, 121, 164, 118, 20, 127, 132, 106, 99, 134, 174, 89, 123, 92, 15, 18, 179, 7, 43, 12, 75, 144, 57, 110, 137, 55, 14, 156, 178, 165, 151, 35, 54, 143, 87, 129, 10, 73, 161, 175, 79, 67, 154, 31, 38, 46, 181, 82, 112, 176, 16, 107, 173, 61]
# print(Crossings(stem4), Nestings(stem4))

print(Crossings(stem1), Nestings(stem1))

for i in range(10):
    # stem1 = [1,2,3,4,5,6,7,8,9,10,11,12]

    stem2 = SimplifyPetalDiagram(stem1.copy())[0]
    # stem2 = list(InsertTrivialPetal(stem1.copy(), ""))[0]

    # print(Gap(stem2))
    print(Crossings(stem2), Nestings(stem2))
    # print("Important:" + str(Gap(stem2) - 2*(Crossings(stem2)[2] + Nestings(stem2)[2] ) - len(stem2)))
    # print(GapMinus(stem2) - 2*(Crossings(stem2) + sum( Nestings(stem2) ) ))
    # print("i: " + str(i))

[2, 2, 4]
[4, 0, 4]
[1, 4, 5]
[4, 0, 4]
[4, 3, 7]
[0, 1, 1]
[336, 252, 588] [287, 246, 533]
[252, 277, 529] [226, 253, 479]
[252, 277, 529] [226, 253, 479]
[252, 277, 529] [226, 253, 479]
[252, 277, 529] [226, 253, 479]
[252, 277, 529] [226, 253, 479]
[252, 277, 529] [226, 253, 479]
[252, 277, 529] [226, 253, 479]
[252, 277, 529] [226, 253, 479]
[252, 277, 529] [226, 253, 479]
[252, 277, 529] [226, 253, 479]


In [ ]:
petey = numpy.random.permutation(17) #[0,3,1,5,7,9,2,6,4,8,10]
print(Gap(petey))
maxDiff = 0
for i in range(10000):
    steve = PetalToStem(petey.copy())
    # print(steve)
    # print(Gap(steve) - Gap(petey))
    # print(Crossings(steve), Nestings(steve))
    tempGap = Gap(steve) - Gap(petey)
    if tempGap > maxDiff:
        maxDiff = tempGap
print(maxDiff)
#Claim: petal to stem conversion always increases gap by at least 2

102
32


In [ ]:
petey = numpy.random.permutation(101)
for i in range(100):
    # petal1 = [11, 2, 12, 5, 10, 3, 1, 8, 6, 0, 9, 7, 4, 13, 14]
    # petal2 = [12, 4, 8, 10, 13, 9, 14, 5, 7, 6, 3, 1, 11, 0, 2]
    # petal1 = [i for i in range(101)]
    # petal2 = [1, 24, 23, 5, 0, 26, 25, 14, 15, 19, 12, 3, 4, 13, 10, 27,28, 7, 6, 9, 8, 11, 20, 18, 21, 17, 16, 22, 2]
    # petal2 = [i for i in range(101)]
    # petal1 = [0, 6, 3, 5, 1, 4, 2]
    # petal2 = [0, 1, 2, 3, 4, 5, 6]
    # petal1 = numpy.random.permutation(201)
    # petal2 = numpy.random.permutation(201)
    petal1 = [0,1,2]
    petal2 = [0,1,2]
    for j in range(1000):
        petal1 = InsertTrivialPetal(petal1.copy(), "")[0]
    for j in range(1000):
        petal2 = InsertTrivialPetal(petal2.copy(), "")[0]
    # petal1 =  RandomlyComplicatePetal([0,1,2], 100)[0]
    # petal2 =  RandomlyComplicatePetal([0,1,2], 100)[0]
    # if len(petal1) > len(petal2):
    #     for i in range(len(petal1 - len(petal2))):
    #         petal2 = InsertTrivialPetal(petal2.copy(), "")[0]


    steve1 = PetalToStem(petal1.copy())
    steve2 = PetalToStem(petal2.copy())
    # print(Gap(steve1) - Gap(steve2))

    num = abs(Crossings(steve1)[2] + Nestings(steve1)[2] - Crossings(steve2)[2] - Nestings(steve2)[2])
    if not ((num <= (len(petal1)*12) - 1)):
        print(num, (num <= (len(petal1)*12) - 1))

print(len(petal1))

24205 False
32234 False
37884 False
33004 False
26103 False
48195 False
44927 False
25197 False
25710 False
37992 False
24625 False
27750 False
2003


In [ ]:
petal1 = numpy.random.permutation(6)
print(Gap(petal1), LeftAndRightGap(petal1), Crossings(petal1), Nestings(petal1), Gap(petal1)-2*Crossings(petal1)[2]-2*Nestings(petal1)[2]-len(petal1))
for i in range(5):

    petal3, trash = InsertTrivialPetal(petal1.copy(), "")

    petal4 = petal3#[petal3[i-len(petal3)] for i in range(2*len(petal3))]

    print(Gap(petal4), LeftAndRightGap(petal4), Crossings(petal4), Nestings(petal4), Gap(petal4)-2*Crossings(petal4)[2]-2*Nestings(petal4)[2]-len(petal4)  )

14 [5, 9] [1, 0, 1] [0, 3, 3] 0
24 [10, 14] [2, 0, 2] [1, 5, 6] 0
22 [8, 14] [1, 0, 1] [1, 5, 6] 0
18 [8, 10] [1, 0, 1] [1, 3, 4] 0
20 [6, 14] [1, 3, 4] [0, 2, 2] 0
24 [8, 16] [1, 5, 6] [1, 1, 2] 0


In [ ]:
#[14, 4, 6, 9, 7, 11, 15, 12, 10, 8, 16, 5, 3, 1, 13, 0, 2]  Mr. Bad Knot

for k in range(1):
    for i in range(25):
        leave = False
        longRandomPermutation, des = RandomlyComplicatePetal(list(numpy.random.permutation( 15 )), 11)
        for i in des:
            if i == "p":
                # print("Yes")
                leave = True
                break
        if leave:
            break
    longRandomPermutation = [14, 4, 6, 9, 7, 11, 15, 12, 10, 8, 16, 5, 3, 1, 13, 0, 2]
    simpleList = []
    orderList = []
    lengthList = []
    reducedOrderList = []        #Important Note: remember to remove this commenting when using several different base permutations
    reducedLengthList = []
    # descriptionList = []


    print("Base Permutation: " + str(longRandomPermutation))
    for i in range(25):
        tempPerm, des = RandomlyComplicatePetal(longRandomPermutation.copy(), 101)
        # simple, description = SimplifyPetalDiagram(tempPerm, False)
        # simple, trash = SimplifyPetalDiagram(simple.copy(), True)
        simple = tempPerm
        simpleList.append(simple)
        orderList.append( DistanceToCountThroughPermutation(simple)//len(simple)  )

        lengthList.append(len(simple))
        # descriptionList.append(description)
        # if gapList[i] != gapList[0]:
        #     print (gapList[i])
        # if descriptionList[i] != descriptionList[0]:
        #     print (descriptionList[i])
    for i in range(len(orderList)):
        if (orderList[i] not in reducedOrderList) or (lengthList[i] not in reducedLengthList):
            reducedOrderList.append(orderList[i])
            reducedLengthList.append(lengthList[i])
    # print(reducedOrderList)
    # print(reducedLengthList)



    for i in range(len(orderList)):
        for j in range(len(orderList)):
            if (orderList[i] >= orderList[j]) and (lengthList[i] < lengthList[j]):
                print(i)
                print(j)
                print(orderList[i])
                print(orderList[j])
                print(lengthList[i])
                print(lengthList[j])
                print(simpleList[i])
                print(simpleList[j])


# print(simpleList)
print(orderList)
print(lengthList)
# print(descriptionList)



Base Permutation: [14, 4, 6, 9, 7, 11, 15, 12, 10, 8, 16, 5, 3, 1, 13, 0, 2]
0
11
62
57
117
125
[88, 36, 49, 50, 1, 111, 112, 85, 84, 86, 87, 40, 4, 3, 66, 67, 43, 2, 48, 47, 78, 77, 58, 19, 20, 59, 98, 101, 56, 27, 28, 72, 71, 26, 91, 90, 25, 52, 39, 63, 57, 75, 115, 114, 97, 64, 65, 105, 110, 62, 61, 96, 89, 76, 70, 82, 8, 92, 93, 9, 100, 99, 23, 22, 11, 12, 83, 6, 5, 34, 35, 60, 116, 33, 44, 45, 32, 55, 38, 37, 13, 14, 102, 103, 17, 73, 74, 95, 94, 16, 104, 113, 15, 7, 54, 53, 31, 18, 21, 30, 79, 0, 108, 109, 69, 107, 106, 68, 46, 51, 80, 81, 24, 41, 42, 29, 10]
[118, 86, 87, 92, 93, 98, 97, 47, 67, 68, 42, 48, 41, 61, 8, 5, 66, 38, 39, 37, 71, 111, 108, 70, 25, 32, 33, 26, 119, 122, 52, 53, 40, 44, 43, 22, 28, 51, 31, 88, 89, 120, 121, 103, 123, 65, 105, 104, 64, 23, 24, 106, 62, 63, 60, 101, 102, 10, 9, 36, 124, 84, 85, 27, 117, 116, 56, 57, 15, 83, 90, 11, 17, 16, 54, 55, 94, 91, 58, 45, 46, 59, 78, 75, 113, 114, 18, 19, 79, 7, 6, 80, 107, 100, 99, 29, 30, 35, 34, 110, 109, 49, 5

In [ ]:
tempPetal = list(numpy.random.permutation(17))
tempStem = PetalToStem(tempPetal,0,0)
print(Crossings(tempStem.copy())[0])
print(TotalPermutationCrossings(tempStem.copy()) )

for i in range(10):
    complicated, trash = RandomlyComplicatePetal(tempStem.copy(), 1, False)
    print(complicated, trash)
    print(Crossings((complicated.copy()))[0])
    print(TotalPermutationCrossings(complicated) )

14
71
[0, 15, 13, 18, 2, 8, 14, 3, 17, 9, 11, 10, 1, 4, 19, 5, 16, 7, 6, 12] Inserted 7 6 at 17, 
19
94
[0, 15, 13, 12, 11, 18, 2, 6, 14, 3, 17, 7, 9, 8, 1, 4, 19, 5, 16, 10] Inserted 12 11 at 3, 
14
96
[0, 15, 13, 18, 4, 8, 14, 5, 3, 2, 17, 9, 11, 10, 1, 6, 19, 7, 16, 12] Inserted 3 2 at 8, 
14
88
[0, 15, 11, 18, 2, 6, 12, 13, 14, 3, 17, 7, 9, 8, 1, 4, 19, 5, 16, 10] Inserted 12 13 at 6, 
14
91
[0, 15, 13, 18, 10, 11, 2, 6, 14, 3, 17, 7, 9, 8, 1, 4, 19, 5, 16, 12] Inserted 10 11 at 4, 
14
95
[0, 13, 11, 16, 2, 6, 12, 3, 15, 18, 17, 7, 9, 8, 1, 4, 19, 5, 14, 10] Inserted 18 17 at 9, 
15
88
[0, 15, 13, 18, 2, 6, 14, 3, 17, 7, 11, 8, 1, 10, 9, 4, 19, 5, 16, 12] Inserted 10 9 at 13, 
20
88
[0, 15, 13, 18, 4, 8, 14, 5, 17, 9, 11, 10, 2, 1, 3, 6, 19, 7, 16, 12] Inserted 2 1 at 12, 
14
94
[0, 15, 13, 18, 2, 6, 14, 3, 17, 7, 9, 8, 1, 4, 12, 11, 19, 5, 16, 10] Inserted 12 11 at 14, 
14
86
[0, 13, 11, 16, 2, 6, 12, 3, 15, 7, 18, 19, 9, 8, 1, 4, 17, 5, 14, 10] Inserted 18 19 at 10, 
14
87


In [ ]:
listOfKnotTypes = []

for i in range(5):
    listOfKnotTypes.append([])
    randomNumber = numpy.random.randint(5,10)
    perm = numpy.random.permutation(2*randomNumber-1)
    for j in range(400):
        tempPerm = perm.copy()
        perm1, trash = RandomlyComplicatePetal(tempPerm, 15)
        listOfKnotTypes[i].append(perm1)

# print(listOfKnotTypes)





In [ ]:
stringy = str(listOfKnotTypes)
newString = stringy.replace("[", "{")
newString1 = newString.replace("]", "}")
print (newString1)

{{{5, 15, 2, 10, 6, 14, 13, 0, 11, 16, 9, 8, 19, 18, 12, 3, 24, 26, 17, 20, 25, 7, 21, 22, 4, 23, 1}, {5, 29, 28, 21, 2, 8, 17, 16, 6, 14, 13, 0, 9, 22, 10, 3, 18, 19, 25, 26, 24, 30, 27, 7, 4, 11, 12, 23, 1, 15, 20}, {13, 29, 5, 6, 8, 22, 11, 12, 1, 2, 20, 19, 14, 21, 18, 34, 33, 0, 25, 30, 26, 38, 37, 9, 4, 16, 17, 3, 32, 27, 28, 36, 23, 24, 35, 15, 10, 31, 7}, {5, 19, 2, 16, 14, 13, 15, 10, 6, 0, 17, 20, 8, 9, 18, 3, 22, 24, 23, 7, 4, 21, 12, 11, 1}, {7, 21, 20, 14, 15, 17, 4, 10, 26, 25, 8, 0, 13, 11, 12, 18, 1, 2, 16, 5, 22, 24, 23, 9, 6, 19, 3}, {7, 21, 2, 19, 18, 6, 5, 12, 10, 9, 15, 16, 8, 0, 17, 24, 23, 22, 20, 3, 26, 14, 29, 30, 13, 28, 27, 11, 4, 25, 1}, {7, 25, 24, 19, 4, 9, 10, 16, 8, 0, 17, 26, 18, 14, 20, 23, 11, 5, 28, 32, 29, 2, 30, 31, 1, 15, 6, 27, 22, 21, 3, 13, 12}, {9, 19, 2, 16, 10, 0, 29, 30, 17, 20, 18, 8, 7, 3, 24, 15, 14, 28, 5, 4, 26, 27, 25, 13, 6, 23, 21, 11, 12, 22, 1}, {11, 29, 30, 27, 12, 13, 1, 4, 21, 20, 8, 18, 16, 0, 28, 31, 19, 32, 23, 24, 26, 14, 1

In [ ]:
def AlteredCountByTwo(petal):
    """Returns the petal counted by two, evens and odds switched from the other"""
    extended = petal.copy()
    extended.extend(petal.copy())
    return [extended[i] for i in range(len(extended)) if i%2 == 1]
def InvariantAttempt(petal):
    return 0
#I need to figure out how to transition the null space back to an invariant

In [ ]:
petal1 = list(numpy.random.permutation(9))

print(petal1, HighestLowestDistance(petal1)[0], HighestLowestDistance(petal1)[1], HighestLowestDistance(AlteredCountByTwo(petal1))[2], 0.5*HighestLowestDistance(petal1)[0] - 0.5*HighestLowestDistance(petal1)[1] + HighestLowestDistance(AlteredCountByTwo(petal1))[2] )
for i in range(10):
    petal2, trash = RandomlyComplicatePetal(petal1.copy(), 5)
    petal2 = [petal2[i-1] for i in range(len(petal2))]
    # petal2 = [petal1[i-1] for i in range(len(petal1))]
    print(petal2, HighestLowestDistance(petal2)[0], HighestLowestDistance(petal2)[1], HighestLowestDistance(AlteredCountByTwo(petal2))[2], .5*HighestLowestDistance(petal2)[0] - .5*HighestLowestDistance(petal2)[1] + HighestLowestDistance(AlteredCountByTwo(petal2))[2] )




[4, 5, 2, 0, 7, 3, 6, 1, 8] 5 8 1 -0.5
[12, 3, 8, 11, 10, 2, 0, 9, 4, 5, 7, 6, 1] -6 0 9 6.0
[12, 3, 6, 2, 8, 7, 14, 13, 0, 11, 10, 9, 4, 5, 1] -2 6 11 7.0
[12, 4, 5, 2, 0, 11, 7, 10, 3, 6, 8, 9, 1] -4 0 8 6.0
[13, 4, 7, 2, 0, 6, 5, 10, 11, 9, 3, 8, 1, 14, 12] 9 13 9 7.0
[18, 4, 12, 13, 5, 2, 11, 10, 16, 7, 6, 9, 0, 17, 3, 8, 14, 15, 1] -12 0 15 9.0
[12, 3, 14, 13, 10, 2, 0, 11, 6, 7, 1, 8, 9, 4, 5] -4 2 10 7.0
[18, 4, 13, 10, 7, 2, 0, 15, 5, 12, 11, 6, 3, 14, 1, 8, 16, 17, 9] -6 0 12 9.0
[12, 3, 8, 2, 0, 11, 10, 5, 4, 9, 6, 7, 1] -4 0 8 6.0
[14, 4, 7, 13, 12, 2, 8, 9, 0, 11, 3, 10, 5, 6, 1] -8 0 11 7.0
[12, 8, 9, 2, 0, 11, 5, 4, 3, 10, 1, 6, 7] -4 0 8 6.0
